In [ ]:
BRANCH = 'r1.13.0'
!pip install inflect
!pip install jsonlines
try:
    # Import NeMo Speech Recognition collection
    import nemo.collections.asr as nemo_asr
except ModuleNotFoundError:
    !python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

# check if we have optional Plotly for visualization
try:
    from plotly import graph_objects as go
except ModuleNotFoundError:
    !pip install plotly
    from plotly import graph_objects as go

# check if we have optional ipywidgets for tqdm/notebook
try:
    import ipywidgets
except ModuleNotFoundError:
    !pip install ipywidgets

# check if CTC beam decoders are installed
try:
    import ctc_decoders
except ModuleNotFoundError:
    # install beam search decoder
    !apt-get install -y swig
    !git clone https://github.com/NVIDIA/NeMo -b "$BRANCH"
    !cd NeMo && bash scripts/asr_language_modeling/ngram_lm/install_beamsearch_decoders.sh
    print('Restarting Colab runtime to successfully import built module.')
    print('Please re-run the notebook.')
    import os
    os.kill(os.getpid(), 9)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
# Import audio processing library
import librosa
# We'll use this to listen to audio
from IPython.display import Audio, display
import gzip
import os, shutil, wget

In [ ]:
#asr_model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_en_conformer_ctc_small")
asr_model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_jasper10x5dr")
#asr_model = nemo_asr.models.EncDecRNNTBPEModel.from_pretrained(model_name="stt_en_conformer_transducer_small")

[NeMo I 2022-12-16 05:57:54 cloud:66] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_jasper10x5dr/versions/1.0.0rc1/files/stt_en_jasper10x5dr.nemo to /root/.cache/torch/NeMo/NeMo_1.13.0/stt_en_jasper10x5dr/856ae08d5c4bd78b5e27f696e96f7aab/stt_en_jasper10x5dr.nemo
[NeMo I 2022-12-16 05:58:34 common:911] Instantiating model from pre-trained checkpoint


[NeMo W 2022-12-16 05:58:56 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    
[NeMo W 2022-12-16 05:58:56 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /data2/vo

[NeMo I 2022-12-16 05:58:56 features:225] PADDING: 16
[NeMo I 2022-12-16 05:59:08 save_restore_connector:243] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.13.0/stt_en_jasper10x5dr/856ae08d5c4bd78b5e27f696e96f7aab/stt_en_jasper10x5dr.nemo.


In [ ]:
files = ['bleeding_13_mesial_all.wav', 'jump_gingival_margin.wav', 'plaque_light_on_all.wav', 'frucation_18_grade_one.wav']
refs = ['bleeding 13 mesial all', 'jump gingival margin', 'plaque light on all', 'frucation 18 grade one']

In [ ]:
asr_model.transcribe(paths2audio_files=files)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

['leading thirteen mesiolingual',
 'jump gingerval margin',
 'plack light on all',
 'forcasion eighteen grade one']

In [ ]:
import inflect
import jsonlines

#could be rewritten in iterators
def make_locations(numbers, advanced_locations):
  locations = [num for num in numbers]
  locations.append('gingival margin')
  for adv_loc in advanced_locations:
    locations.append(adv_loc)
  for num in numbers:
    for adv_loc in advanced_locations:
      locations.append(f'{num} {adv_loc}')
  return locations


def make_conditions(numbers, locations):
  conditions = ['bleeding']
  for plaque_cond in ['light', 'medium', 'severe']:
    conditions.append(f'plaque {plaque_cond}')
  for num in numbers:
    for loc in locations:
      # maybe locations used in wrong way here
      conditions.append(f'furcation {loc} grade {num}')
  return conditions

def make_texts(actions, locations, conditions):
  for loc in locations:
    for act in actions:
      yield f'{act} {loc}'
    for cond in conditions:
      yield f'{cond} {loc}'

actions = ['jump']
p = inflect.engine()
numbers = [' '.join(p.number_to_words(i).split('-')) for i in range(1, 33)]
advanced_locations = ["mesial", "distal", "all"]
locations = make_locations(numbers, advanced_locations)
conditions = make_conditions(numbers, locations)

with jsonlines.open('kenlm_train_texts.json', 'w') as f:
  for lm_text in make_texts(actions, locations, conditions):
    f.write({"audio_filepath": "", "text": lm_text})


In [ ]:
!source NeMo/scripts/asr_language_modeling/ngram_lm/install_beamsearch_decoders.sh

In [ ]:
#for char-based add to train_kenlm.py "discount_arg = "--discount_fallback"" on #126 row 

!python NeMo/scripts/asr_language_modeling/ngram_lm/train_kenlm.py --nemo_model_file /root/.cache/torch/NeMo/NeMo_1.13.0/stt_en_jasper10x5dr/856ae08d5c4bd78b5e27f696e96f7aab/stt_en_jasper10x5dr.nemo \
                          --train_file ./kenlm_train_texts.json \
                          --kenlm_bin_path /content/decoders/kenlm/build/bin \
                          --kenlm_model_file ./dentist_kenlm_jasper \
                          --ngram_length 3

[NeMo W 2022-12-16 06:16:06 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo I 2022-12-16 06:16:10 train_kenlm:87] Loading nemo model '/root/.cache/torch/NeMo/NeMo_1.13.0/stt_en_jasper10x5dr/856ae08d5c4bd78b5e27f696e96f7aab/stt_en_jasper10x5dr.nemo' ...
[NeMo W 2022-12-16 06:16:23 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    
[NeMo W 

In [ ]:
beam_search_lm = nemo_asr.modules.BeamSearchDecoderWithLM(
    vocab=list(asr_model.decoder.vocabulary),
    beam_width=32,
    alpha=0, beta=1,
    lm_path='dentist_kenlm_jasper',
    num_cpus=max(os.cpu_count(), 1),
    input_tensor=False)

In [ ]:
# softmax implementation in NumPy
def softmax(logits):
    e = np.exp(logits - np.max(logits))
    return e / e.sum(axis=-1).reshape([logits.shape[0], 1])

# let's do inference once again but without decoder
logits = asr_model.transcribe(files, logprobs=True)[3]
probs = softmax(logits)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def infer_lm_beamsearch(asr_model, files, beam_search_lm):
  logits = asr_model.transcribe(files, logprobs=True)
  probs = [softmax(logit) for logit in logits]
  beam_preds = [beam_search_lm.forward(log_probs = np.expand_dims(prob, axis=0), log_probs_length=None)[0][0] for prob in probs]
  for file, pred in zip(files, beam_preds):
    print(file, pred)

infer_lm_beamsearch(asr_model, files, beam_search_lm)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

bleeding_13_mesial_all.wav (-28.124488830566406, 'bleeding thirteen mesial')
jump_gingival_margin.wav (-14.972404479980469, 'jump gingival margin')
plaque_light_on_all.wav (-9.337037086486816, 'plaque light one all')
frucation_18_grade_one.wav (-5.7196269035339355, 'furcation eighteen grade one')


In [ ]:
beam_search_lm.forward(log_probs = np.expand_dims(probs, axis=0), log_probs_length=None)

[[(-5.7196269035339355, 'furcation eighteen grade one'),
  (-12.946653366088867, 'furcation eighteen grade on'),
  (-14.621949195861816, 'furcation eighteen grade one '),
  (-21.632577896118164, 'furcation eighteen grade o'),
  (-22.346908569335938, 'furcation eighteen grade n'),
  (-22.67534637451172, 'furcation eighteen grade one e'),
  (-22.964330673217773, 'furcation eighteen grade one n'),
  (-23.067916870117188, 'furcation eighteen grade one o'),
  (-23.552400588989258, 'furcation eighteen grade one m'),
  (-23.67625617980957, 'furcation eighteen grade one d'),
  (-23.73257827758789, 'furcation eighteen grade one a'),
  (-23.798383712768555, 'furcation eighteen grade e'),
  (-23.805051803588867, 'furcation eighteen grade one t'),
  (-23.897430419921875, 'furcation eighteen grade one s'),
  (-24.638586044311523, 'furcation eighteen grade one l'),
  (-24.911352157592773, 'furcation eighteen grade one g'),
  (-25.189529418945312, 'furcation eighteen grade one f'),
  (-25.87345886230